### 1. Importing libraries
For this assignment I've used the following libraries:
- VTK
- ipywidgets
- IPython
- plotly

If these libraries are not present, see [Python's package installation guide](https://packaging.python.org/en/latest/tutorials/installing-packages/).

In [87]:
import vtk
from ipywidgets import *
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

### 2. Extracting useful information from the .vti file
For this task we require the following information:
1. Range of data values (for slider and histogram)
2. Coordinates and corresponding values (for isosurface)

I've used standard VTK functions to get the above.

In [88]:
# Reads the .vti file

reader = vtk.vtkXMLImageDataReader()

#########################################################
# Make sure the file is present in the path given below #
filepath = "./mixture.vti"                              #
#########################################################

reader.SetFileName(filepath)
reader.Update()
data = reader.GetOutput()

In [89]:
# Gets the range in form of minimum and maximum values inside the data

arr = data.GetPointData().GetArray('ImageFile')
minVal, maxVal = arr.GetRange()[0], arr.GetRange()[1]
print(minVal, maxVal)

-0.9935540556907654 0.43280163407325745


In [90]:
totalPoints = data.GetNumberOfPoints()

values = [] # scalar values
x = [] # x coordinates
y = [] # y coordinates
z = [] # z coordinates

for i in range(totalPoints):
    values.append(data.GetPointData().GetScalars().GetValue(i))
    coords = data.GetPoint(i)
    x.append(coords[0])
    y.append(coords[1])
    z.append(coords[2])

In [91]:
# Checking that the extracted values are equal and aligned

print(len(x),len(y),len(z),len(values))

421875 421875 421875 421875


### 3. Creating the UI elements
Next step is to create the slider and the button. For function description and parameter details, see the [widget API](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html) of ipywidgets.

In [92]:
# Creates the slider 

slider = FloatSlider(value=0.0,
                        min=minVal,
                        max=maxVal,
                        step=0.01,
                        description='Isovalue:',
                        disabled=False,
                        continuous_update=False,
                        orientation='horizontal',
                        readout=True,
                        readout_format='.2f',
                        layout=Layout(width='40%')
                    )

display(slider)

FloatSlider(value=0.0, continuous_update=False, description='Isovalue:', layout=Layout(width='40%'), max=0.432…

In [93]:
# Creates the button

button = Button(description='Reset',
                    button_style='',
                    layout=Layout(width='15%')
               )

button

Button(description='Reset', layout=Layout(width='15%'), style=ButtonStyle())

### 4. Creating the isosurface and the histogram
- For isosurface, I've used [plotly's graph objects API for isosurface](https://plotly.github.io/plotly.py-docs/generated/plotly.graph_objects.Isosurface.html)
- For histogram, I've used [plotly express API for histogram](https://plotly.github.io/plotly.py-docs/generated/plotly.express.histogram.html)

In [94]:
# Creates isosurface figure

surf = go.Figure(data=go.Isosurface(x=x,
                                    y=y,
                                    z=z,
                                    value=values,
                                    isomin=0.00,
                                    isomax=0.00,
                                    cmin=minVal,
                                    cmax=maxVal,
                                    showscale=False,
                                    colorscale='Plasma'
                                    )
                )

# Hides tick labels from the x, y, z axes
_ = surf.update_layout(scene=dict(xaxis=dict(showticklabels=False),
                                  yaxis=dict(showticklabels=False),
                                  zaxis=dict(showticklabels=False)
                                 )
                      )

In [95]:
# Creates histogram figure

hist = go.Figure(px.histogram(x=values,
                              nbins=30
                             )
                )

# Add axis labels
_ = hist.update_layout(xaxis_title='Vortex scalar values',
                       yaxis_title='Frequency'
                      )

### 5. Defining the update functions for interactions
There are 4 functions that may get triggered on interactions with slider and button:
1. ***updateSurface(isoval)*** updates the isosurface.
2. ***updateHist(isoval)*** updates the histogram based on session and isovalue.
3. ***onSliderChange(change)*** is called when the slider value is changed.
4. ***onButtonClick(_)*** is called on button click. '_' is a [throwaway variable](https://stackoverflow.com/questions/5893163/what-is-the-purpose-of-the-single-underscore-variable-in-python).

In [96]:
# Function that updates the isosurface 

def updateSurface(isoval):
    surf.update_traces(isomin=isoval)
    surf.update_traces(isomax=isoval)
    return surf

In [97]:
# Function that updates the histogram 

def updateHist(isoval):
    global startSession
    if startSession and isoval == 0.00:
        startSession = False
        hist.update_traces(x=values) # use full range at start of session
    else:
        hist.update_traces(x=[val for val in values if val >= isoval-0.25 and val <= isoval+0.25])
    return hist

In [98]:
# Defines what to do on slider change

def onSliderChange(change):
    surf = updateSurface(change.new)
    hist = updateHist(change.new)
    plots = HBox([go.FigureWidget(surf), go.FigureWidget(hist)])
    clear_output()
    display(ui, plots)
    
# Bind slider with the function above
slider.observe(onSliderChange, names='value')

#### Handling the 'Reset' button
The following boolean variable *startSession* keeps track of the Reset button. The idea used here is that I'm treating every interaction by the user as part of one 'session'. So initially, *startSession* is True. After the plots are updated because of slider change, *startSession* becomes False because it's an old session. When the reset button is clicked, a new session begins and *startSession* becomes True again. 

This variable also helps in updating the histogram's x axis depending on whether the reset button has been pressed or not.

In [99]:
# Defining and initializing the session variable 

startSession = True

In [100]:
# Defines what happens on Reset button press

def onButtonClick(_):
    global startSession
    # begin new session
    startSession = True
    # set slider value to 0
    slider.value = 0.00

# Bind button with the function above
button.on_click(onButtonClick)

### 6. Setting the display elements
- Slider and button make up the UI. 
- Isosurface and histogram make up the plots. 
Plotly figures aren't directly compatible with HBox, so I've used [FigureWidget](https://plotly.com/python/figurewidget/) to first convert the figures into widgets. This allows figures to be treated as widgets and arrange them as we like.

In [101]:
# Put slider and button in one horizontal box and the plots into another

ui = HBox([slider, button])
plots = HBox([go.FigureWidget(surf), go.FigureWidget(hist)])

In [102]:
# Calling this function will start the visualization

def beginViz():
    display(ui, plots)
    slider.value = 0.00

## ~~~~~~~START~~~~~~~~

In [103]:
beginViz()

    'data': [{'cmax': 0.43280163407325745,
              'cmin': -0.993554055690…